**Defining Foreign Keys:**

A foreign key in SQL is a *table-level construct* that constrains *one or more columns* in that table to only allow values that are *present in a different set of columns*, typically but not always located on a different table.

the columns which they are constrained towards the referenced columns

The referenced columns *almost always define the primary key* for their owning table, though there are exceptions to this. 

connects together pairs of rows which have a relationship with each other // ??? pairs of rows
and SQLAlchemy assigns very deep importance to this concept in virtually every area of its operation.


The single column foreign key is more common, and at the column level is specified by constructing a *ForeignKey* object as an argument to a Column object:

In [22]:
from sqlalchemy import URL ,MetaData, Table , Column , Integer ,String , ForeignKey , create_engine

url = URL.create(
    drivername="postgresql",
    username="postgres",
    host="192.168.0.8",
    port="5432",
    password="1234",
    database="tests"
)

engine = create_engine(url)
metadata_obj = MetaData()#schema="public"

user = Table(
    "user",
    metadata_obj,
    Column("user_id", Integer, primary_key=True),
    #schema="public"
)

user_preference = Table(
    "user_preference",
    metadata_obj,
    Column("pref_id", Integer, primary_key=True),
    Column("user_id", Integer, ForeignKey("user.user_id"), nullable=False),#ForeignKey("public.user.user_id")
    Column("pref_name", String(40), nullable=False),
    Column("pref_value", String(100)),
    #schema="public"
)


metadata_obj.create_all(engine)

The argument to ForeignKey is most commonly a string of the form {tablename}.{columnname}, or for a table in a remote schema or “owner” of the form {schemaname}.{tablename}.{columnname}.

It may also be an actual Column object ad it's accessed from an existing Table object via its c collection:

In [26]:
ForeignKey(user.c.user_id)


user_test = Table(
    "user_test",
    metadata_obj,
    Column("pref_id", Integer, primary_key=True),
    Column("user_id", Integer, ForeignKey(user.c.user_id), nullable=False),
    Column("pref_name", String(40), nullable=False),
    Column("pref_value", String(100)),
)

user_test.create(engine)

The *advantage to using a string* is that the in-python linkage between user and user_preference is resolved only when first needed, so that *table objects can be easily spread across multiple modules and defined in any order.*

Foreign keys may also be defined at the table level, using the *ForeignKeyConstraint* object. This object can describe a single- or multi-column foreign key. A multi-column foreign key is known as a *composite foreign key*, and *almost always* references a table that has a composite primary key

In [29]:
invoice = Table(
    "invoice",
    metadata_obj,
    Column("invoice_id", Integer, primary_key=True),
    Column("ref_num", Integer, primary_key=True),
    Column("description", String(60), nullable=False),
)

from sqlalchemy import ForeignKeyConstraint

invoice_item = Table(
    "invoice_item",
    metadata_obj,
    Column("item_id", Integer, primary_key=True),
    Column("item_name", String(60), nullable=False),
    Column("invoice_id", Integer, nullable=False),
    Column("ref_num", Integer, nullable=False),
    ForeignKeyConstraint(
        ["invoice_id", "ref_num"], ["invoice.invoice_id", "invoice.ref_num"]
    ),
)

InvalidRequestError: Table 'public.invoice' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.